In [1]:
import pandas as pd
import math

In [2]:
wine_raw = pd.read_csv('winemag-data-130k-v2.csv', index_col=0)

In [3]:
wine_raw.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [4]:
wine_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129971 entries, 0 to 129970
Data columns (total 13 columns):
country                  129908 non-null object
description              129971 non-null object
designation              92506 non-null object
points                   129971 non-null int64
price                    120975 non-null float64
province                 129908 non-null object
region_1                 108724 non-null object
region_2                 50511 non-null object
taster_name              103727 non-null object
taster_twitter_handle    98758 non-null object
title                    129971 non-null object
variety                  129970 non-null object
winery                   129971 non-null object
dtypes: float64(1), int64(1), object(11)
memory usage: 13.9+ MB


In [5]:
wine_df = wine_raw.loc[:,['country', 'points']]

In [6]:
americas = ('US', 'Chile', 'Argentina', 'Canada', 'Uruguay', 'Mexico', 'Brazil', 'Peru')

europe = ('France', 'Italy', 'Spain', 'Portugal','Austria','Germany',
          'Greece', 'Hungary', 'Bulgaria', 'Romania','Turkey', 'Slovenia', 
          'Georgia', 'England', 'Croatia', 'Moldova', 'Morocco', 'Ukraine',
          'Czech Republic', 'Macedonia', 'Serbia', 'Cyprus',
          'Switzerland', 'Luxembourg', 'Bosnia and Herzegovina', 'Armenia','Slovakia')

In [7]:
wine_df = (wine_df.assign(americas = wine_df['country']
                  .apply(lambda x : 1 if x in americas else 0))
          )
wine_df = (wine_df.assign(europe = wine_df['country']
                  .apply(lambda x : 1 if x in europe else 0))
          )
wine_df = (wine_df.assign(others = wine_df['country']
                  .apply(lambda x : 1 if x not in americas and x not in europe else 0))
          )

In [8]:
wine_df.head()

,country,points,americas,europe,others
0,Italy,87,0,1,0
1,Portugal,87,0,1,0
2,US,87,1,0,0
3,US,87,1,0,0
4,US,87,1,0,0


In [9]:
import matplotlib.pyplot as plt

plt.figure(figsize=[14,7])

plt.hist(wine_df[wine_df.americas == 1]['points'], 
         color = 'red', 
         alpha = 0.5,
         label = 'American')

plt.axvline(wine_df[wine_df.americas == 1]['points'].mean(),
            color='red')

plt.hist(wine_df[wine_df.europe == 1]['points'], 
         color='blue', 
         alpha=.5, 
         label = 'European')

plt.axvline(wine_df[wine_df.europe == 1]['points'].mean(),
            color='blue')

plt.hist(wine_df[wine_df.others == 1]['points'], 
         color='yellow', 
         alpha=.5, 
         label = 'Other')

plt.axvline(wine_df[wine_df.others == 1]['points'].mean(),
            color='yellow')

plt.xlabel('Ratings')
plt.ylabel('Total count of ratings')
plt.title('Wine ratings')
plt.legend()

plt.show()

<Figure size 1400x700 with 1 Axes>

$H_0$: If the America's wine is better that all other wins, then it will have a higher mean $\mu$ rating. $\alpha$ = 0.05

$H_A$: If the America's wine is not better, then it will have an equal or lower mean $\mu$ rating. $\mu\neq\mu_0$.

In [10]:
import scipy.stats as stats

stats.ttest_1samp(wine_df[wine_df.americas == 1]['points'], wine_df[wine_df.americas != 1]['points'].mean())

Ttest_1sampResult(statistic=-23.094458180972676, pvalue=1.616045046579982e-117)

In [11]:
american = wine_df[wine_df.americas == 1]['points']
others = wine_df[wine_df.americas != 1]['points']

In [12]:
cohensD = abs(american.mean() - others.mean()) / math.sqrt(american.std()**2 + others.std()**2)
cohensD

0.06754611701470721